In [1]:
# evaluate multinomial logistic regression model
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression


url="https://raw.githubusercontent.com/ASIF-Mahmud1/Exploration/MLlessons/Lesson%201/Logistice%20Regression/emailDataset.csv"
from io import StringIO
import string
import pandas as pd
import requests
s=requests.get(url).text

message_data=pd.read_csv(StringIO(s))
message_data.head()


,emailId,tag,mail-message
0,1,boss,"Dear Tom, I can't thank you enough for forward..."
1,2,possy,"Hi Linda, How's it going? Sorry I haven't been..."
2,3,possy,"Hey, Susi, whats wrong with you? Why aren't yo..."
3,4,boss,"Dear Barylior, I will need your opinion on how..."
4,5,boss,"Dear Liz,I really appreciate your understandin..."


In [2]:
message_data.groupby('tag').describe()


emailId                                              
        count  mean       std  min   25%   50%    75%   max
tag                                                        
boss     10.0   9.0  5.374838  1.0  5.25   8.0  13.25  17.0
possy    10.0  12.0  6.324555  2.0  8.50  12.5  17.25  20.0

In [3]:
message_data_copy = message_data['mail-message'].copy()

message_data_copy

0     Dear Tom, I can't thank you enough for forward...
1     Hi Linda, How's it going? Sorry I haven't been...
2     Hey, Susi, whats wrong with you? Why aren't yo...
3     Dear Barylior, I will need your opinion on how...
4     Dear Liz,I really appreciate your understandin...
5     Dear Chris,I just wanted to drop you a note to...
6     Dear Ryanna, Thank you so much for the generou...
7     Brian ! like to talk to people from other part...
8     Dear Ms. Smith, This is to bring to your notic...
9     Hey Lucie, since we last talked, I started dat...
10    Hello Sir, I am a student of yours from UCLA. ...
11    Emy . we are not friends anymore. On my Birthd...
12    Hi Weasel! Hope this mail finds you in the bes...
13    Dear Sir,  Please accept this mail as a notice...
14    Hi my friend� I haven�t seen you for a lot tim...
15    Dear Ms. Lee, I am writing to request consider...
16    Dear Mr./Ms. {Recipient�s Name}, I am writing ...
17    Liza, Being your sister is the greatest bl

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

In [5]:
def text_preprocess(text):
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = [word for word in text.split() if word.lower() not in stopwords.words('english')]
    return " ".join(text)

In [6]:
def stemmer (text):
    text = text.split()
    words = ""
    for i in text:
            stemmer = SnowballStemmer("english")
            words += (stemmer.stem(i))+" "
    return words

In [7]:
message_data_copy = message_data_copy.apply(text_preprocess)
message_data_copy = message_data_copy.apply(stemmer)    # New Line
# message_data_copy

In [8]:
vectorizer = TfidfVectorizer("english")
message_mat = vectorizer.fit_transform(message_data_copy)


c:\users\kids\appdata\local\programs\python\python38\lib\site-packages\sklearn\utils\validation.py:68: FutureWarning: Pass input=english as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


In [9]:
# message_train, message_test, spam_nospam_train, spam_nospam_test = train_test_split(message_mat, 
#                                                         message_data['tag'], test_size=0.3, random_state=20)

message_train, message_test, spam_nospam_train, spam_nospam_test = train_test_split(message_mat, 
                                                        message_data['tag'], random_state=20)

In [10]:
Spam_model = LogisticRegression(multi_class='multinomial', solver='lbfgs')
Spam_model.fit(message_train, spam_nospam_train)
pred = Spam_model.predict(message_test)
print("The prediction is :", pred)

The prediction is : ['possy' 'possy' 'possy' 'possy' 'possy']


In [11]:
dummyEmail = ["Hey, Susi, whats wrong with you? Why aren't you "]
dummyEmailMatrix= vectorizer.transform(dummyEmail) 
dummyEmailMatrix
predict= Spam_model.predict(dummyEmailMatrix)
print("The prediction is :", predict)

The prediction is : ['possy']


In [12]:
prob_Of_Each_Class = Spam_model.predict_proba(dummyEmailMatrix)
# summarize the predicted probabilities
print('Predicted Probabilities: %s' % prob_Of_Each_Class[0])

Predicted Probabilities: [0.30738675 0.69261325]
